<font size="5"><b>Изучение развития игровой идустрии в первой половине XXI века</b></font>

- Автор: Смирнова Анастасия
- Дата: 11.01.2025

### Цели и задачи проекта

<font color='#777778'>**Цель проекта** - собрать данные для написания статьи-исследования по особенностям развития игровой индустрии для популяризации игры "Секреты Темнолесья".

<font color='#777778'>**Задачи проекта:**
- Загрузить данные и ознакомиться с ними;
- Проверить ошибки и провести предобработку данных;
- Провести фильтрацию данных, отобрав для анализа информацию об играх с 2000 по 2013 год ключительно;
- Категоризовать данные по оценкам пользователей и оценкам критиков.
</font>

### Описание данных

<font color='#777778'>Данные `datasets/new_games.csv` содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:
- `Name` — название игры.
- `Platform` — название платформы.
- `Year of Release` — год выпуска игры.
- `Genre` — жанр игры.
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).
- `EU sales` — продажи в Европе (в миллионах проданных копий).
- `JP sales` — продажи в Японии (в миллионах проданных копий).
- `Other sales` — продажи в других странах (в миллионах проданных копий).
- `Critic Score` — оценка критиков (от 0 до 100).
- `User Score` — оценка пользователей (от 0 до 10).
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.</font>

### Содержимое проекта
<font color='#777778'>
1. Загрузка и знакомство с данными.
2. Проверка ошибок в данных и их предобработка.
<font color='#777778'>
- Названия, или метки, столбцов датафрейма;
- Наличие пропусков в данных;
- Типы данных;
- Явные и неявные дубликаты в данных.
</font>
3. Фильтрация данных.
4. Категоризация данных.
5. Итоговый вывод



## 1. Загрузка и знакомство с данными.

In [1]:
# импортируем библиотекку pandas
import pandas as pd
pd.options.mode.chained_assignment = None

#импортируем библиотеку numpy
import numpy as numpy

In [2]:
#загружаем данные датасета
games = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

In [3]:
#сохраняем длину исходного датафрейма как "games_initial" для сравнения с итоговым после преобразований
games_initial = len(games)

In [4]:
#выводи информацию о датафрейме
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [5]:
#выводим первые строки датафрейма на экран
games.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN



<font color='#777778'>В полученном датафрейме - 11 столбцов и 16 956 строк. По количеству непустых строк видно, что в некоторых столбцах есть пропуски: **`Name`, `Year of Release`, `Genre`, `Critic Score`, `User Score`, `Rating`.** Далее необходимо проанализировать пропуски детально и принять решение о том, как с ними поступить при анализе данных.

<font color='#777778'>Наименования и количество столбцов соответствуют заявленным в описании данных.
Для удобства работы со столбцами датафрейма необходимо привести написание наименований столбцов в соответствии с snake case.

<font color='#777778'>В датафрейме данные представлены двумя типами - **float64** и **object**.
Поскольку в столбце **`Year`** представлены данные о годе выпуска игр, необходимо представить данные целочисленным типом **int64**, а не дробным **float64**.

<font color='#777778'>Для данных в столбцах **`EU Sales`** и **`JP sales`** более корректным будет использование типа **float64** по аналогии со столбцами **`NA sales`** и **`Other Sales`**, поскольку значения в данных столбцах измеряются в числовом формате.

Аналогично преобразования из **object** требует столбец **`User Score`**, поскольку данные в нем представлены числом от 0 до 10. По аналогии со столцом **`Critic Score`** преобразуем данные в столбце к типу **float64**.</font>



## 2. Проверка ошибок в данных и их предобработка.

### *Названия, или метки, столбцов датафрейма*

In [6]:
#выводи исходные названия столбцов на экран
games.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [7]:
#приводим названия столбцов к нижнему регистру
games.columns = games.columns.str.lower()

#заменяем пробелы на _
games.columns = games.columns.str.replace(' ','_')

In [8]:
#выводим наименование столбов после преобразования для проверки
games.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

<font color='#777778'>Выполнили преобразование наименований столбцов датафрейма в соответствии со стилем snake case.

### *Типы данных*

<font color='#777778'>Нам необходимо преобразовать тип данных в следующих столбцах:
- **`year_of_release`** из float64 в int64;
- **`eu_sales`** и **`jp_sales`** из object в float64;
- **`user_score `** из object в float64 </font>

<font color='#777778'> Для того, чтобы выяснить, почему столбцы **`eu_sales`** и **`jp_sales`** и **`user_score`**  имеют нечисловой тип данных выведем уникальные значения этих столбцов на экран.

In [9]:
#выводим уникальные значения столбца eu_sales
games['eu_sales'].unique()

array(['28.96', '3.58', '12.76', '10.93', '8.89', '2.26', '9.14', '9.18',
       '6.94', '0.63', '10.95', '7.47', '6.18', '8.03', '4.89', '8.49',
       '9.09', '0.4', '3.75', '9.2', '4.46', '2.71', '3.44', '5.14',
       '5.49', '3.9', '5.35', '3.17', '5.09', '4.24', '5.04', '5.86',
       '3.68', '4.19', '5.73', '3.59', '4.51', '2.55', '4.02', '4.37',
       '6.31', '3.45', '2.81', '2.85', '3.49', '0.01', '3.35', '2.04',
       '3.07', '3.87', '3.0', '4.82', '3.64', '2.15', '3.69', '2.65',
       '2.56', '3.11', '3.14', '1.94', '1.95', '2.47', '2.28', '3.42',
       '3.63', '2.36', '1.71', '1.85', '2.79', '1.24', '6.12', '1.53',
       '3.47', '2.24', '5.01', '2.01', '1.72', '2.07', '6.42', '3.86',
       '0.45', '3.48', '1.89', '5.75', '2.17', '1.37', '2.35', '1.18',
       '2.11', '1.88', '2.83', '2.99', '2.89', '3.27', '2.22', '2.14',
       '1.45', '1.75', '1.04', '1.77', '3.02', '2.75', '2.16', '1.9',
       '2.59', '2.2', '4.3', '0.93', '2.53', '2.52', '1.79', '1.3', '2.6',
   

In [10]:
#выводим уникальные значения столбца jp_sales
games['jp_sales'].unique()

array(['3.77', '6.81', '3.79', '3.28', '10.22', '4.22', '6.5', '2.93',
       '4.7', '0.28', '1.93', '4.13', '7.2', '3.6', '0.24', '2.53',
       '0.98', '0.41', '3.54', '4.16', '6.04', '4.18', '3.84', '0.06',
       '0.47', '5.38', '5.32', '5.65', '1.87', '0.13', '3.12', '0.36',
       '0.11', '4.35', '0.65', '0.07', '0.08', '0.49', '0.3', '2.66',
       '2.69', '0.48', '0.38', '5.33', '1.91', '3.96', '3.1', '1.1',
       '1.2', '0.14', '2.54', '2.14', '0.81', '2.12', '0.44', '3.15',
       '1.25', '0.04', '0.0', '2.47', '2.23', '1.69', '0.01', '3.0',
       '0.02', '4.39', '1.98', '0.1', '3.81', '0.05', '2.49', '1.58',
       '3.14', '2.73', '0.66', '0.22', '3.63', '1.45', '1.31', '2.43',
       '0.7', '0.35', '1.4', '0.6', '2.26', '1.42', '1.28', '1.39',
       '0.87', '0.17', '0.94', '0.19', '0.21', '1.6', '0.16', '1.03',
       '0.25', '2.06', '1.49', '1.29', '0.09', '2.87', '0.03', '0.78',
       '0.83', '2.33', '2.02', '1.36', '1.81', '1.97', '0.91', '0.99',
       '0.95', '2.0'

<font color='#777778'> Среди столбцов **`eu_sales`** и **`jp_sales`** встречаются значения `unknown` свидетельствующие о том, что для некоторых игр отсутсвует информация о продажах в указанных странах. Это говорит о том, что скорее всего некоторые игры выпускались только под рынок конкретной страны и не продавались по всему миру.

In [11]:
#выводим уникальные значения user_score
games['user_score'].unique()

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

<font color='#777778'> В столбце **`user_score `**  встречаются пропуски, а также значения `'tbd' (to be determined)`, означающие, что пользовательская оценка для игры еще не определена.

<font color='#777778'> Мы можем заменить значения `unknown` и `tbd` на пропуски и далее оценить масштаб проблемы для выбора способа их обработки.

In [12]:
#выполняем преобразования и выводим данные о датафрейме на экран

games['year_of_release'] = games['year_of_release'].astype('Int64')

for column in ['eu_sales','jp_sales','user_score']:
  games[column] = pd.to_numeric(games[column], errors = 'coerce')

games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  Int64  
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16950 non-null  float64
 6   jp_sales         16952 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: Int64(1), float64(6), object(4)
memory usage: 1.4+ MB


<font color='#777778'>Теперь все столбцы имеют правильный тип данных.

### *Наличие пропусков в данных*

In [13]:
#выводим количество пропусков в данных
empty_share = pd.DataFrame(games.isna().sum())

In [14]:
#находим долю пропущенных значений
empty_share['share'] = games.isna().sum() / len(games) * 100
empty_share = empty_share[(empty_share['share'] > 0)]
empty_share.sort_values(by = 'share',ascending = False).style.background_gradient(cmap='YlOrRd', axis=0, subset = 'share')

,0,share
user_score,9268,54.659118
critic_score,8714,51.391838
rating,6871,40.522529
year_of_release,275,1.621845
eu_sales,6,0.035386
jp_sales,4,0.023590
name,2,0.011795
genre,2,0.011795


<font color='#777778'>Пропуски встречаются в 6 столбцах:

- **`user_score`** - количество пропусков также существенно (более 50% или 	9268 значений) и для принятия решения потребуется изучить причины пропусков более детально;
- **`critic_score`** - количество пропусков (8 714) значительно и составляет более половины всех значений. Поскольку данный столбец необходим для дальнейшей категоризации данных, необходимо более детальное изучение возможных причин пропусков;
-**`rating`** - в данном столбце также пропущенно более 40% значений(6 871 пропуск). Поскольку рассчитать рейтинг ERSB самостоятельно невозможно, наиболее оптимальной стратегией будет заполнение пропусков значением-индикатором **not stated**, которое будет обозначать, что для данных игр информация о рейтинге ESRB отсутсвует. Для понимания причин отсутствия рейтинга ESRB также можно провести более детальный анализ данных;
-**`year_of_release`**- 275 пропущенных значения (около 1,62%). Можно предположить, что отсутствие данных о годе выпуска связано с технической ошибкой при заполнении даннных. Так как пропуски составляют менее 2%, данные их также можно исключить, поскольку они в любом случае будут отсеяны при отборе данных за период с 2000 по 2013 год в связи с отусутствием информации и годе выпуска;
- **`eu_sales`** - 6 пропущенных значений(0.03%). Появление данных пропусков связано с преобразование столбца к типу **`float64`** и заменой значений `unknown` на пропуски. По всей видимости некоторые игры не продавались в Европе, поэтому данные о продажах в этом регионе отсутствуют. Количество пропусков крайне мало, поэтому эти строки можно удалить из датафрейма без искажения результата анализа;
- **`jp_sales`** - 4 пропущенных значения (0.02%). Появление пропусков связано с обработкой аналогичной **`eu_sales`**. Для данных игр отсутствуют данные по продажам в Японии, скорее всего они там не продавались. Количество пропусков крайне мало, поэтому эти строки можно удалить из датафрейма без искажения результата анализа;
- **`name`** - 2 пропущенных значения (менее 0.02%).Скорее всего появление пропусков связано с ошибкой в заполнении данных. Данные пропуски на точность расчетов не повлияют, их можно удалить из данных;
-**`genre`** - 2 пропущенных значения (менее 0.02%). Скорее всего появление пропусков связано с ошибкой в заполнении данных. Так как количество пропусков несущественно, можем также исключить их из данных.
 </font>

In [15]:
#удаляем строки с пропусками с столбцах name, year_of_release, genre, eu_sales, jp_sales
games = games.dropna(subset = [
    'name',
    'year_of_release',
    'genre',
])

In [16]:
#заменяем пропуски в eu_sales и jp_sales при помощи transform
games[['jp_sales','eu_sales']] = games[['jp_sales','eu_sales']].fillna(games.groupby(['platform', 'year_of_release'])[['jp_sales','eu_sales']].transform('mean'))

<font color='#777778'>Изучим пропуски в `user_score` и `critic_score` детально. Для этого создадим срез данных по играм, где значения в этих столбцах отсутствуют.

In [17]:
#создадим срез строк, где есть пропуски в user_score или в critic_score.
empty_scores_df = games[(games['user_score'].isna() == True) | (games['critic_score']).isna() == True].copy()

In [18]:
#найдем суммарное количество продаж по каждой игре
empty_scores_df['total_sales'] = empty_scores_df[['na_sales','eu_sales','jp_sales','other_sales']].sum(axis = 1)

In [19]:
#отсортируем значения по убыванию total_sales и оценим, какие значения по продажам присутствуют в датафреме.
empty_scores_df.sort_values(by = 'total_sales',ascending = False)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN,40.24
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN,31.38
5,Tetris,GB,1989,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN,30.26
9,Duck Hunt,NES,1984,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN,28.31
10,Nintendogs,DS,2005,Simulation,9.05,10.95,1.93,2.74,NaN,NaN,NaN,24.67
...,...,...,...,...,...,...,...,...,...,...,...,...
16452,DoDonPachi Saidaioujou,X360,2013,Shooter,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01
16453,Last Escort: Club Katze,PS2,2010,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01
16955,Winning Post 8 2016,PSV,2016,Simulation,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01
16950,SCORE International Baja 1000: The Official Game,PS2,2008,Racing,0.00,0.00,0.00,0.00,NaN,NaN,NaN,0.00


<font color='#777778'>Видно, что значения **`user_score`** и  **`critic_score`** отсутствуют как для игр, где количество проданных копий равняется нулю, так и для тех, где количество проданных копий исчисляется несколькими десятками миллионов. Примечательно, что довольно часто пропуски встречаются в обоих столбцах одновременно.


<font color='#777778'>Найдем минимальное и максимальное значение в датафрейме в срезе `empty_scores_df`. Оценим также медиану и среднее значение.

In [20]:
#находим минимальное значение
empty_scores_df['total_sales'].min()

0.0

In [21]:
#находим максимальное значение
empty_scores_df['total_sales'].max()

40.24

In [22]:
#находим медианное значние
empty_scores_df['total_sales'].median()

0.11

In [23]:
#находим среднее значение
empty_scores_df['total_sales'].mean()

0.36456169061341565

<font color='#777778'>В результате сравнения видим, что минимальное значение total_sales составляет - 0, а максимальное - 40.24 млн. проданных копий.
При этом медианное и среднее значение в наборе данных говорят о том, что для большинства игр количество проданных копий составляет менее 1 млн., что свидетельствует о низкой популярности игр этих игр. Найдем количество игр, количество проданных копий для которых составляет менее 1 млн. Вполне возможно, что таки игры не получали ни оценку пользователей ни оценку критиков, так как остались незамеченными относительно более популярных игр. </font>

<font color='#777778'> Найдем количество игр, для которых количество продаж составляет до 1 млн. копий в количественном и процентном выражении.

In [24]:
#присвоим категорию при помощи pd.cut()
empty_scores_df['category_by_sales'] = pd.cut(empty_scores_df['total_sales'],
                                       bins = [0,1,42],
                                       labels = ["менее 1 млн.", "более 1 млн."],right = False)

In [25]:
#подсчитаем количество игр в каждой категории
empty_scores_grouped = pd.DataFrame(empty_scores_df.groupby('category_by_sales', observed = True)['name'].count())
empty_scores_grouped['share'] = empty_scores_grouped / empty_scores_df.shape[0] * 100
empty_scores_grouped

,name,share
category_by_sales,,
менее 1 млн.,8953,92.298969
более 1 млн.,747,7.701031


<font color='#777778'>Количество игр с пропусками в столбцах **`user_score`** и  **`critic_score`** и количеством продаж до 1 млн. копий составляет более 92%. Скорее всего эти пропуски неслучайны и связаны с низкой известностью как среди пользователей, так и среди критиков. Пропуски для таких игр оставим без изменений. Тогда мы сможем выделить их в отдельную группу при дальнейшей категоризации.

Количество игр c пропусками в **`user_score`** и  **`critic_score`**, для которых продажи составили более 1 млн. проданных копий, составляет почти 8%. Пропущенные значения для таких игр можно заполнить средними арифметическим в рамках `genre` и `total_sales`.</font>

In [26]:
#находим total_sales для датафрейма games
games['total_sales'] = games[['na_sales','eu_sales','jp_sales','other_sales']].sum(axis = 1)

In [27]:
#создаем функцию для заполнения пропусков user_score
def replace_score_user(x):
  if pd.isna(x['user_score']):
    if x['total_sales'] < 1:
       return None
    else:
      group = games[(games['genre'] == x['genre']) & (games['total_sales'] == games['total_sales'])]
      return round(group['user_score'].mean(),2)
  else:
    return x['user_score']

#применяем функцию
games['user_score'] = games.apply(replace_score_user,axis = 1)

In [28]:
#создаем функцию для заполнения пропусков critic_score
def replace_score_user(x):
  if pd.isna(x['critic_score']):
    if x['total_sales'] < 1:
       return None
    else:
      group = games[(games['genre'] == x['genre']) & (games['total_sales'] == games['total_sales'])]
      return round(group['critic_score'].mean(),2)
  else:
    return x['critic_score']

#применяем функцию
games['critic_score'] = games.apply(replace_score_user,axis = 1)

<font color='#777778'>Попробуем определить природу пропусков в столбце `rating`.

In [29]:
#создадим срез строк с пропусками в столбце rating
empty_rating_df = games[(games['rating'].isna() == True)].copy()

In [30]:
#изучим, на каких платформах выпускались игры, для которых отсутствует информация о рейтинге ESRB
empty_rating_df['platform'].unique()

array(['NES', 'GB', 'DS', 'SNES', 'GBA', 'PS4', '3DS', 'N64', 'X360',
       'Wii', '2600', 'PS', 'XOne', 'GC', 'PS3', 'GEN', 'PC', 'PSP',
       'WiiU', 'PS2', 'XB', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

<font color='#777778'> Можно заметить, что большинство значений в списке представлены японскими игровыми платформами. Можно предположить, что некоторые игры выпускались только для японского рынка и не получили американский рейтинг ESRB.

In [31]:
#создаем функцию для заполнения пропусков в столбце rating
games['rating'] = games['rating'].fillna('not stated')

In [32]:
#проверяем обработку пропусков в датафрейме
empty_share_changed = pd.DataFrame(games.isna().sum())

In [33]:
#вычисляем процент пропущенных значений в столбцах user_score и ctitic_score
empty_share_changed['share'] = games.isna().sum() / len(games) * 100
empty_share_changed.sort_values(by = 'share', ascending = False).style.background_gradient(cmap='YlOrRd', axis=0, subset = 'share')

,0,share
user_score,8427,50.524612
critic_score,7873,47.203070
name,0,0.000000
platform,0,0.000000
year_of_release,0,0.000000
genre,0,0.000000
na_sales,0,0.000000
eu_sales,0,0.000000
jp_sales,0,0.000000
other_sales,0,0.000000


<font color='#777778'> Снизили количество пропусков в столбцах **`user_score`** и  **`critic_score`**, заполнив их средними значениями в рамках жанра и суммарного объема проданных копий для тех игр, объем продаж по которым превышает более 1 млн. копий. Для игр, где суммарный объём продаж составляет менее 1 млн. копий, оставили пропуски без изменений.

<font color='#777778'> Все пропущенные значения в датафрейме обработаны. Можно приступать к обработке дубликатов.

### *Явные и неявные дубликаты в данных*

<font color='#777778'>Выведем уникальные значения в столбцах **`genre`**, **`platform`** **`rating`**,**`year_of_release`**

In [34]:
#выводим уникальные значения genre
numpy.sort(games['genre'].unique())

array(['ACTION', 'ADVENTURE', 'Action', 'Adventure', 'FIGHTING',
       'Fighting', 'MISC', 'Misc', 'PLATFORM', 'PUZZLE', 'Platform',
       'Puzzle', 'RACING', 'ROLE-PLAYING', 'Racing', 'Role-Playing',
       'SHOOTER', 'SIMULATION', 'SPORTS', 'STRATEGY', 'Shooter',
       'Simulation', 'Sports', 'Strategy'], dtype=object)

<font color='#777778'>В столбце **`genre`** есть явные дубликаты, появление которых связано с написанием жанров в различном регистре. Приведем все жанры к нижнему регистру и проведём повторную проверку наличия дубликатов в данном столбце.</font>

In [35]:
#приводим написание к нижнему регистру
games['genre'] = games['genre'].str.lower()

<font color='#777778'>Проводим повторную проверку наличия дубликатов

In [36]:
numpy.sort(games['genre'].unique())

array(['action', 'adventure', 'fighting', 'misc', 'platform', 'puzzle',
       'racing', 'role-playing', 'shooter', 'simulation', 'sports',
       'strategy'], dtype=object)

<font color='#777778'>Дубликаты в столбце  **`genre`** устранены

In [37]:
#выводим уникальные значения platform
numpy.sort(games['platform'].unique())


array(['2600', '3DO', '3DS', 'DC', 'DS', 'GB', 'GBA', 'GC', 'GEN', 'GG',
       'N64', 'NES', 'NG', 'PC', 'PCFX', 'PS', 'PS2', 'PS3', 'PS4', 'PSP',
       'PSV', 'SAT', 'SCD', 'SNES', 'TG16', 'WS', 'Wii', 'WiiU', 'X360',
       'XB', 'XOne'], dtype=object)

<font color='#777778'>В столбце **`platform`** дубликаты отсутствуют. Наименования соответствуют существующим игровым платформам.

In [38]:
#выводим уникальные значения rating
numpy.sort(games['rating'].unique())

array(['AO', 'E', 'E10+', 'EC', 'K-A', 'M', 'RP', 'T', 'not stated'],
      dtype=object)

<font color='#777778'>В столбце **`rating`** есть неявные дубликаты в паре **`E`**  - **`K-A`** .

После 1998 года рейтинг **`K-A` (kids to adult)** перестал использоваться и был заменен на рейтинг **`E`**.
Приведем значения к общепринятому стандарту, заменив значения **`K-A`**  на **`E`**.</font>

In [39]:
#заменяем значения "K-A" на "E"
games['rating'] = games['rating'].str.replace('K-A','E')

In [40]:
#проверяем успешность замены
numpy.sort(games['rating'].unique())

array(['AO', 'E', 'E10+', 'EC', 'M', 'RP', 'T', 'not stated'],
      dtype=object)

<font color='#777778'>Замена неявных дубликатов в столбце  **`rating`** произведена успешно.

In [41]:
#выводим уникальные значения year_of_release
numpy.sort(games['year_of_release'].unique())

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016])

<font color='#777778'>В столбце **`year_of_release`** дубликаты отсутствуют

In [42]:
#выводим информацию о получившемся датафрейме

games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16679 entries, 0 to 16955
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16679 non-null  object 
 1   platform         16679 non-null  object 
 2   year_of_release  16679 non-null  Int64  
 3   genre            16679 non-null  object 
 4   na_sales         16679 non-null  float64
 5   eu_sales         16679 non-null  float64
 6   jp_sales         16679 non-null  float64
 7   other_sales      16679 non-null  float64
 8   critic_score     8806 non-null   float64
 9   user_score       8252 non-null   float64
 10  rating           16679 non-null  object 
 11  total_sales      16679 non-null  float64
dtypes: Int64(1), float64(7), object(4)
memory usage: 1.7+ MB


In [43]:
#сравниваем количество строк в исходном и получившемся датафрейме
games_initial - games.shape[0]

277

In [44]:
#находим, какая часть строк в процентном соотношении была удалена из датафрейма при его обработке
(games_initial - games.shape[0]) / games_initial * 100

1.6336400094361876

*Вывод по обработке датафрейма*

<font color='#777778'>В результате очистки датафрейма от пропусков, обработки дубликтов и изменения типа данных исходный датафрейм стал на 277 строк (1,63%) меньше своего изначального объема. Такая потеря в данных несущественна и не скажется на достоверности результатов анализа.

<font color='#777778'>Были измененены типы данных в следующих столбцах:

- **`year_of_release`** - тип данных преобразован к `int64` благодаря чему год отображается корректно без лишней точки и нуля;
- **`eu_sales`** и **`jp_sales`** - тип данных преобразован к  `float64`, что позволяет производить математические вычисления с данными столбцами, а также при необходимости найти суммарный объем продаж, сложив данные со столбцами **`na_sales`** и **`other_sales`**;
- **`user_score`** - тип данных преобразован к `float64`, благодаря чему со столбцом можно производить математические вычисления. </font>

<font color='#777778'>Были произведены следующие преобразования с пропусками:
- **`name`** - удалены 2 строки с пропусками;
- **`year_of_release`** - удалено 275 строк с пропусками;
- **`genre`** - удалены 2 строки с пропусками;
- **`eu_sales`** - 6 пропущенных значения заменены в рамках платформы и года выпуска;
- **`jp_sales`** - 4 пропущенных значения заменены в рамках платформы и года выпуска;
- **`critic_score`** и **`user_score`**  - 8 953 пропуска оставлено без изменений, а 747 пропусков замнено на среднюю оценку в рамках жанра игры и суммарного количества проданных копий;
- **`rating`** - заменено 6 871 значений на индикатор "not stated".</font>

<font color='#777778'>Были найдены и обработаны дубликаты в следующих столбцах:

- **`genre`** - явные дубликаты, связанные с написанием жанра в различном регистре. Преобразование к нижнему регистру позволило избавиться от дубликатов;
- **`rating`**- неявные дубликаты в паре `K-A` - `E`. Устаревшее значение `K-A` было заменено на `E`.</font>

Все необходимые преобразования выполнены. Можно приступать к следующему этапу.</font>




## 3. Фильтрация данных

<font color='#777778'>Для дальнейшего анализа и категоризации нам потребуется отобрать данные об играх, выпущенных с 2000 по 2013 год включительно.

In [45]:
#создаем срез данных
games_filtered = games[(games['year_of_release'] >= 2000) & (games['year_of_release'] <= 2013)].copy()

In [46]:
#проверяем, что в выборку попали нужные нам годы
numpy.sort(games_filtered['year_of_release'].unique())

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013])

In [47]:
#выведем информацию по полученном срезе
games_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12980 entries, 0 to 16954
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             12980 non-null  object 
 1   platform         12980 non-null  object 
 2   year_of_release  12980 non-null  Int64  
 3   genre            12980 non-null  object 
 4   na_sales         12980 non-null  float64
 5   eu_sales         12980 non-null  float64
 6   jp_sales         12980 non-null  float64
 7   other_sales      12980 non-null  float64
 8   critic_score     7543 non-null   float64
 9   user_score       6836 non-null   float64
 10  rating           12980 non-null  object 
 11  total_sales      12980 non-null  float64
dtypes: Int64(1), float64(7), object(4)
memory usage: 1.3+ MB


<font color='#777778'>Срез данных создан успешно. В полученном срезе - 12 979 строк.

## 4. Категоризация данных

### *Категоризация по оценкам пользователей*

<font color='#777778'>Для категоризации по оценкам пользователей применим метод pd.cut(), разделив все игры на категории в соответствии со следующими ограничениями:

- `высокая оценка` (от 8 до 10 включительно);
- `средняя оценка` (от 3 до 8, не включая правую границу интервала);
- `низкая оценка` (от 0 до 3, не включая правую границу интервала);
- `без оценки` - если данные об оценке отсутствуют.</font>

In [48]:
#создадим функцию для категоризации игр по оценкам пользователей
def user_rating_class(x):
    if pd.isna(x['user_score']):
      return 'Без оценки'
    else:
      if x['user_score'] >= 0 and x['user_score'] < 3:
        return 'Низкая оценка'
      elif x['user_score'] >= 3 and x['user_score'] < 8:
        return 'Средняя оценка'
      else:
        return 'Высокая оценка'

In [49]:
#применим функцию
games_filtered['user_rating'] = games_filtered.apply(user_rating_class, axis = 1)

In [50]:
#посчитаем, какое количество игр вошло в каждую группу
result_user_rating = pd.DataFrame(games_filtered.groupby('user_rating',observed = True)['user_rating'].count())
result_user_rating['share'] = result_user_rating['user_rating'] / games_filtered.shape[0] * 100
result_user_rating.sort_values(by = 'share',ascending = False).style.background_gradient(cmap='YlOrRd', axis=0, subset = 'share')

,user_rating,share
user_rating,,
Без оценки,6144,47.334361
Средняя оценка,4412,33.990755
Высокая оценка,2307,17.773498
Низкая оценка,117,0.901387


<font color='#777778'> Почти половина рассматриваемых игр не получала пользовательскую оценку. Это говорит о том, что идустрия в период с 2000 по 2013 год была очень разнообразна, но очень большое количество игр не набрало широкой мировой известности.
<font color='#777778'>
Большинство игр, получишвих оценку пользователей, попадает в категорию "Средняя оценка". Количество игр, получивших высокую оценку почти вдвое меньше. Современных игроков достаточно непросто впечталить.</font>

<font color='#777778'>Изучение разбивки игр по жанрам и платформам в категории "Высокая оценка" сможет дать более глубокое представление о том, какие игры пользуются популярностью у пользователей.

### *Категоризация по оценкам критиков*

<font color='#777778'>Для категоризации игр по оценкам критиков также используем метод  pd.cut().
Деление произведем по следующим параметрам:

- `высокая оценка` (от 80 до 100 включительно);
- `средняя оценка` (от 30 до 80, не включая правую границу интервала);
- `низкая оценка` (от 0 до 30, не включая правую границу интервала). </font>

In [51]:
#создадим функцию для категоризации игр по оценкам критиков
def critic_rating_class(x):
    if pd.isna(x['critic_score']):
      return 'Без оценки'
    else:
      if x['critic_score'] >= 0 and x['critic_score'] < 30:
        return 'Низкая оценка'
      elif x['critic_score'] >= 30 and x['critic_score'] < 80:
        return 'Средняя оценка'
      else:
        return 'Высокая оценка'

In [52]:
#применим функцию
games_filtered['critic_rating'] = games_filtered.apply(critic_rating_class, axis = 1)

In [53]:
#посчитаем, какое количество игр вошло в каждую группу
result_critic_rating = pd.DataFrame(games_filtered.groupby('critic_rating',observed = True)['critic_rating'].count())
result_critic_rating['share'] = result_critic_rating / games_filtered.shape[0] * 100
result_critic_rating.sort_values(by = 'share',ascending = False).style.background_gradient(cmap='YlOrRd', axis=0, subset = 'share')

,critic_rating,share
critic_rating,,
Средняя оценка,5776,44.499230
Без оценки,5437,41.887519
Высокая оценка,1712,13.189522
Низкая оценка,55,0.423729


<font color='#777778'>
Почти 42% рассматривамых игр не получали оценку критиков.

<font color='#777778'>Большинство игр получили среднюю оценку, в то время как высокую оценку получило всего 13 процентов, рассматриваемых игр.

<font color='#777778'>Такое распределение говорит о том, что критики требовательны при проставлении оценок и получить высокую оценку игре непросто.

Более детальное изучение игр в разбивке по жанрам и платформам в категории "Высокая оценка" и сравнение результатов с аналогичным анализом игр по оценкам пользователей позволит составить более точное представление о том, какие игры считаются успешными в рамках современной индустрии XXI века.</font>

### *Топ-7 Платформ по количеству игр*

<font color='#777778'>Для того, чтобы выделить топ-7 платформ по количеству игр для начала произведем группировку при помощи метода `groupby()`, отсортируем результат по убыванию и выведем первые семь строк получившегося результата.

In [54]:
#находим количество игр для каждой платформы
top_7_platforms_count = games_filtered.groupby('platform',as_index = False)['name'].count()

In [55]:
top_7_platforms_pivot = pd.pivot_table(games_filtered, values = 'name', index = 'platform', aggfunc = 'count')
top_7_platforms_pivot.sort_values(by = 'name',ascending = False,).head(7)

,name
platform,
PS2,2154
DS,2146
Wii,1294
PSP,1199
X360,1138
PS3,1107
GBA,826


In [56]:
#сортируем полученный результат по убыванию
sorted_result = pd.DataFrame(top_7_platforms_count.sort_values(by = 'name', ascending = False))

In [57]:
#выводим первые семь строк на экран
sorted_result.head(7)

,platform,name
9,PS2,2154
2,DS,2146
15,Wii,1294
12,PSP,1199
17,X360,1138
10,PS3,1107
4,GBA,826


<font color='#777778'>Наибольшее количество игр вышло на платформах `PS2`  и `DS`. Разрыв между количеством игр на этих платформах невелик и составляет всего 12 игр.

С третьего по шестое место рапсоложились платформы `Wii`, `PSP`, `X360` и `PS3`, количество игр на которыих колеблется в диапазоне от 1106 до 1292.</font>

<font color='#777778'>На седьмом месте расположилась платформа `GBA`. Разрыв с шестым местом довольно существенен и составляет 281 игру.

## 5. Итоговый вывод

<font color='#777778'>В результате проделанной работы исходный датафрейм в `16 956` строк был преобразован в датафрейм размером `16 679` строк. Были выполнены следующие преобразования:

- Обработаны пропуски,
- Обработаны дубликаты,
- Изменены типы данных столбцов. </font>

<font color='#777778'>Далее при помощи фильтрации был получен срез в `12 980` строк. Были отобраны данные об играх, выпущенных с 2000 по 2013 год включительно.

<font color='#777778'> Категоризация игр по оценкам пользователей показала, что большинство игр попадает в категорию "Без оценки". Это говорит о том, что игровая индустрия в рассматриваемом периоде, предоставляла широкий выбор игр, однако много из них не получило широкого распространения. Это может быть связано с распространием игр только на локальных рынках.

Большинство игр получило среднюю оценку. Количество игр с высокой оценкой вдвое меньше. </font>

<font color='#777778'>Более детальный анализ игр в категории с высокой оценкой позволит составить полное впечатление о факторах, влияющих на рейтинг пользователей и критиков.

<font color='#777778'>Наиболее популярными игровыми платформами оказались PS2 и DS.
Можно предположить, что более высокая популярность PS2 по сравнению с PS3 связаны с рассматриваемым временным периодом. PS2 была выпущена в 2000 году, в то время как PS3 была выпущена на 6 лет позже. Примечательно, что несмотря на выход новой консоли компания Sony все же продолжила выпускать игры для PS2 в достаточно большом количестве.

Важно также отметить, что среди топ-7 платформ не оказалось `PC`. Это говорит о том, что популярность игровых консолей среди геймеров сравнительно выше чем ПК.</font>